# Correct the ARCS labels

The labels from the KITTI data I downloaded were in the format:<br>
dz, dy, dx, x, y, z, yaw<br>
The ARCS labels seemed to be in the order:<br>
dz, dy, dx, x, z, y, yaw<br>
And the x, z, y coordinates were rotated 90 degrees counterclockwise. I'm going to make a second set of labels that is in the same format as KITTI.

Interesting. So right off the bat, my function is plotting the bounding boxes in a different spot that the point clouds (I'm working with lidar). I think this is because of how the previous person converted the data from the labeler. Will you be able to help me correct the labels?

So I have a bunch of .txt files and each one corresponds to a lidar frame. They look like this:
Car 0.00 0 -2.49 0.00 0.00 0.00 0.00 1.97 2.74 5.16 -29.63 -1.78 0.27 0.50
Car 0.00 0 -2.66 0.00 0.00 0.00 0.00 2.02 2.71 4.95 -30.38 -0.21 26.84 5.20
Car 0.00 0 -2.65 0.00 0.00 0.00 0.00 2.13 2.48 5.36 -28.66 -0.11 29.51 5.21
Motorcycle 0.00 0 2.22 0.00 0.00 0.00 0.00 1.30 1.19 2.58 6.27 0.43 24.76 3.77

So, I basically ignore the first 8 values (the type of car and the 7 numerical values) when I'm just drawing 3d bounding boxes.

## ACTUALLY- THE DIFFERENCE BETWEEN ARCS AND KITTI (not the raw data I was working with) IS A NEGATIVE YAW, so I'll just do that.

In [30]:
import os
import numpy as np
from pathlib import Path

In [31]:
DATA_DIR_ROOT = '../data'

In [32]:
def get_labels(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = []
            bbox.append(parts[0])
            bbox = bbox + [float(value) for value in parts[1:15]]  # Extract the bounding box dimensions and location
            labels.append(bbox)
    return np.array(labels)

In [33]:
def save_labels(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            # Join each element of the label list with a space and write to the file
            file.write(' '.join(map(str, label)) + '\n')

In [ ]:
# Go into the ARCS data folder
labels_dir_path = Path(DATA_DIR_ROOT, 'labels')
# Create correct ARCS data folder
corrected_dir_labels_path = Path(DATA_DIR_ROOT, 'corrected_labels')
corrected_dir_labels_path.mkdir(exist_ok=True)
# For each .txt file
for label_file in labels_dir_path.iterdir():
    if str(label_file).endswith('.txt'):
        # Get the names of the file
        # Get the base filename ('006428.txt')
        filename = os.path.basename(label_file)
        # Split the filename from the extension ('006428', '.txt')
        file_id, extension = os.path.splitext(filename)
        new_file_location = Path(corrected_dir_labels_path, filename)
        
        # Make a list of corrected labels
        corrected_labels_list = []
        # For each label in the file
        labels = get_labels(label_file)
        # print(labels)
        for label in labels:
            label[-1] = str(-float(label[-1]))
        # Save the new labels to a new file in the corrected ARCS data folder
        save_labels(new_file_location, labels)
    

[]
[]
[]
[]
[]
[]
[]
[]
[['Car' '0.0' '0.0' '2.43' '0.0' '0.0' '0.0' '0.0' '1.94' '2.73' '4.49'
  '-27.1' '-2.09' '-6.66' '0.55']]
[['Car' '0.0' '0.0' '2.44' '0.0' '0.0' '0.0' '0.0' '1.96' '2.7' '4.56'
  '-26.18' '-2.02' '-6.1' '0.55']]
[['Car' '0.0' '0.0' '2.46' '0.0' '0.0' '0.0' '0.0' '1.99' '2.68' '4.63'
  '-25.27' '-1.95' '-5.54' '0.55']]
[['Car' '0.0' '0.0' '2.48' '0.0' '0.0' '0.0' '0.0' '2.01' '2.65' '4.69'
  '-24.35' '-1.88' '-4.98' '0.55']]
[['Car' '0.0' '0.0' '2.51' '0.0' '0.0' '0.0' '0.0' '2.03' '2.63' '4.76'
  '-23.43' '-1.82' '-4.42' '0.55']]
[['Car' '0.0' '0.0' '2.55' '0.0' '0.0' '0.0' '0.0' '2.05' '2.6' '4.83'
  '-22.52' '-1.75' '-3.86' '0.55']]
[['Car' '0.0' '0.0' '2.59' '0.0' '0.0' '0.0' '0.0' '2.08' '2.58' '4.9'
  '-21.6' '-1.68' '-3.3' '0.55']]
[['Car' '0.0' '0.0' '2.65' '0.0' '0.0' '0.0' '0.0' '2.1' '2.56' '4.97'
  '-20.69' '-1.61' '-2.75' '0.55']]
[['Car' '0.0' '0.0' '2.73' '0.0' '0.0' '0.0' '0.0' '2.12' '2.53' '5.04'
  '-19.77' '-1.54' '-2.19' '0.55']]
[['Car' '0.0